Code for running LAMMPS in a python loop to generate DNN training data

## Importing Libraries

In [1]:
import random
import itertools as it
import string
import os

## Running LAMMPS

In [2]:
alphabet_list = list(string.ascii_uppercase)
all_atoms_seperate = list(it.product(alphabet_list, repeat = 2))
all_atoms = []
for i in range(len(all_atoms_seperate)):
    atom = all_atoms_seperate[i][0] + all_atoms_seperate[i][1]
    all_atoms.append(atom)
    
#all_atoms is a list which contains all possible two letter combinations, which is the list which will be used
#for the bead names

In [4]:
inp = open("base.in", "r")
inp_lines = inp.readlines()
inp.close

<function TextIOWrapper.close()>

In [5]:
for i in range(len(inp_lines)):
    print('{}: '.format(i) + inp_lines[i])

0: processors 1 1 1

1: units real

2: atom_style atomic

3: 

4: region box block 0 20 0 20 0 20

5: 

6: 

7: 

8: 

9: pair_style table linear 1000

10: 

11: 

12: velocity all create 1 157

13: 

14: neighbor 0.7 bin

15: neigh_modify every 20 delay 0 check no 

16: 

17: fix 1 all nvt temp 300 300 100

18: dump id all atom 50 dump.lammpstrj

19: 

20: timestep 1

21: 

22: run 100000


In [3]:
def write_input(beadnumber, filenumber):
    inp = open("base.in", "r")
    inp_lines = inp.readlines()
    inp.close
    
    create_atoms = ''
    create_atom_seed = random.randint(1,1000000)
    
    masses = ''
    
    potentials = ''
    
    AtomList = []
    for i in range(beadnumber):
        AtomList.append(all_atoms[i])
        
    AtomPairs = list(it.combinations_with_replacement(AtomList, r = 2))
    
    vel_seed = random.randint(1,1000000)
    
    for j in range(1, beadnumber + 1):
        create_atoms = create_atoms + 'create_atoms {} random 20 {} box \n'.format(j, create_atom_seed + j)
        masses = masses + 'mass {} 500 \n'.format(j)
        
    for i in range(len(AtomPairs)):
        a = AtomList.index(AtomPairs[i][0]) + 1
        b = AtomList.index(AtomPairs[i][1]) + 1
        potentials = potentials + "pair_coeff {} {} ".format(a, b) + \
        "LAMMPS_Potentials/{}beadpotential{}.{}_{}.table\
        ".format(beadnumber, filenumber, AtomPairs[i][0],AtomPairs[i][1]) + \
        " {}_{} \n".format(AtomPairs[i][0], AtomPairs[i][1])
    
    
    inp_lines[5] = "create_box {} box \n".format(beadnumber)
    
    inp_lines[6] = create_atoms
    inp_lines[7] = masses
    inp_lines[10] = potentials
    
    inp_lines[12] = "velocity all create 1 {} \n".format(vel_seed)
    
    
    dumpfile = 'LAMMPS_Trajectories/{}bead{}.dcd'.format(beadnumber, filenumber)
    inp_lines[18] = 'dump id all dcd 50 ' + dumpfile
    
    
    out = open('LAMMPS_Inputs/{}bead{}.in'.format(beadnumber, filenumber), 'w')
    for obj in inp_lines:
        out.write(obj)
    out.close
    
#creates an input file based on a template file "base.in"
#that uses the previously created random potentials

In [4]:
write_input(1,1)

In [5]:
os.system("lmp -in LAMMPS_Inputs/1bead1.in")

0

In [8]:
for i in range(1,5):
    for j in range(1):
        write_input(i,j)
        os.system("lmp -in LAMMPS_Inputs/{}bead{}.in".format(i,j))